# Preparación de Datasets de entrenamiento para Predicciones

## Predicciones i columnas necessarias

### Predicción de la Posición Final en Carrera

**Objetivo**: Predecir la posición final de un piloto en una carrera.

**Tablas necesarias**:

- **Identificadores**:
    - `raceId`: ID carrera
    - `driverId`: ID piloto
    - `constructorId`: ID escudería
    - `circuitId`: ID circuita
    - `year`: Temporada
    - `round`: Número de ronda en la temporada

- **Variable Objetivo**:
    - `positionOrder`: Posición final en la carrera

- **Variables predictivas**
    - `qualifying_position`: Posición en la parrilla
    - `gap_to_pole`: Diferencia con el tiempo de la pole
    - `driver_age`: Edad del piloto
    - `standing_before`: Puntos en el campeonato
    - `wins_before`: Número de victorias
    - `constructor_points_before`: Puntos del equipo constructor
    - `constructor_wins_before`: Número de victorias de la escudería
    - `country`: País donde se encuentra
    - `altitude`: Altitud sobre el nivel del mar
    - `driver_avg_position_at_circuit`: Posición promedio del piloto en el circuito
    - `driver_best_position_at_circuit`: Mejor posición histórica del piloto en el circuito
    - `driver_races_at_circuit`: Veces que el piloto ha corrido aquí.

In [7]:
import pandas as pd
from datetime import datetime

# Cargar todos los datasets
circuits = pd.read_csv('../data/original/circuits.csv')
constructor_standings = pd.read_csv('../data/original/constructor_standings.csv')
constructors = pd.read_csv('../data/original/constructors.csv')
driver_standings = pd.read_csv('../data/original/driver_standings.csv')
drivers = pd.read_csv('../data/original/drivers.csv')
qualifying = pd.read_csv('../data/original/qualifying.csv')
races = pd.read_csv('../data/original/races.csv')
results = pd.read_csv('../data/original/results.csv')

import pandas as pd
from datetime import datetime

# 1. Preparar datos de carreras con información de circuito
races_circuits = pd.merge(races, circuits, on='circuitId', how='left')

# 2. Calcular edad del piloto en el momento de cada carrera - CORRECCIÓN AQUÍ
drivers['dob'] = pd.to_datetime(drivers['dob'], errors='coerce')  # Aseguramos formato datetime
results_with_driver = pd.merge(results, drivers[['driverId', 'dob', 'nationality']], on='driverId', how='left')
races_circuits['date'] = pd.to_datetime(races_circuits['date'], errors='coerce')  # Aseguramos formato datetime

full_data = pd.merge(results_with_driver, races_circuits, on='raceId', how='left')

# Calculamos edad correctamente verificando que ambas fechas sean válidas
full_data['driver_age'] = (full_data['date'] - full_data['dob']).dt.days / 365.25

# 3. Unir datos de clasificación
qualifying_agg = qualifying.groupby(['raceId', 'driverId']).agg({
    'position': 'first',
    'q1': 'first',
    'q2': 'first',
    'q3': 'first'
}).reset_index().rename(columns={'position': 'qualifying_position'})

full_data = pd.merge(full_data, qualifying_agg, on=['raceId', 'driverId'], how='left')

# 4. Calcular gap to pole - CON MANEJO DE VALORES NULOS
# Primero convertimos los tiempos a segundos para evitar problemas con formatos
def time_to_seconds(time_str):
    try:
        if pd.isna(time_str):
            return None
        m, s = time_str.split(':')
        return int(m) * 60 + float(s)
    except:
        return None

qualifying['q3_seconds'] = qualifying['q3'].apply(time_to_seconds)
pole_positions = qualifying[qualifying['position'] == 1][['raceId', 'q3_seconds']].rename(columns={'q3_seconds': 'pole_time_seconds'})
full_data = pd.merge(full_data, pole_positions, on='raceId', how='left')
full_data['q3_seconds'] = full_data['q3'].apply(time_to_seconds)
full_data['gap_to_pole'] = full_data['q3_seconds'] - full_data['pole_time_seconds']

# 5. Unir datos de standings antes de la carrera
# Ordenamos por raceId para asegurar el shift correcto
driver_standings = driver_standings.sort_values(['driverId', 'raceId'])
constructor_standings = constructor_standings.sort_values(['constructorId', 'raceId'])

driver_standings['standing_before'] = driver_standings.groupby(['driverId'])['points'].shift(1)
driver_standings['wins_before'] = driver_standings.groupby(['driverId'])['wins'].shift(1)

constructor_standings['constructor_points_before'] = constructor_standings.groupby(['constructorId'])['points'].shift(1)
constructor_standings['constructor_wins_before'] = constructor_standings.groupby(['constructorId'])['wins'].shift(1)

full_data = pd.merge(full_data, 
                    driver_standings[['raceId', 'driverId', 'standing_before', 'wins_before']],
                    on=['raceId', 'driverId'], how='left')

full_data = pd.merge(full_data,
                    constructor_standings[['raceId', 'constructorId', 'constructor_points_before', 'constructor_wins_before']],
                    on=['raceId', 'constructorId'], how='left')

# 6. Calcular estadísticas históricas por piloto-circuito
def calculate_historical_stats(df):
    df = df.sort_values('date')
    historical = df.groupby(['driverId', 'circuitId']).agg({
        'positionOrder': ['mean', 'min', 'count'],
        'points': 'sum'
    }).reset_index()
    
    historical.columns = ['driverId', 'circuitId', 
                        'driver_avg_position_at_circuit', 
                        'driver_best_position_at_circuit',
                        'driver_races_at_circuit',
                        'driver_points_at_circuit']
    
    return historical

historical_stats = calculate_historical_stats(full_data)
full_data = pd.merge(full_data, historical_stats, on=['driverId', 'circuitId'], how='left')

# 7. Seleccionar columnas finales
final_columns = [
    # Identificadores
    'raceId', 'driverId', 'constructorId', 'circuitId', 'year', 'round',
    
    # Variable objetivo
    'positionOrder',
    
    # Datos de clasificación
    'qualifying_position', 'gap_to_pole',
    
    # Datos piloto
    'driver_age', 'standing_before', 'wins_before',
    
    # Datos equipo
    'constructor_points_before', 'constructor_wins_before',
    
    # Datos circuito
    'country', 'alt',
    
    # Históricos
    'driver_avg_position_at_circuit', 'driver_best_position_at_circuit',
    'driver_races_at_circuit'
]

final_dataset = full_data[final_columns].copy()

# 8. Limpieza final
final_dataset = final_dataset.dropna(subset=['positionOrder'])  # Eliminar DNFs si queremos solo posiciones
final_dataset = final_dataset[final_dataset['positionOrder'] > 0]  # Solo posiciones válidas

# Guardar dataset final
final_dataset.to_csv('../data/train/f1_position_prediction_dataset.csv', index=False)